# Quick start look into sBART

The sBART pipeline provides the user with all the necessary pieces to construct a template matching algorithm, by selecting
the Stellar and Telluric templates, the underlying RV model and the method with which new, tentative RVs, are given to the model.

In this Section we showcase a small example of building a RV model from the different parts of SBART.

**Note:** it is highly advised to not run SBART inside jupyter notebooks, as one might run into issues due to  terminal logging.

## Specifying the input and output paths

**Note**: the disk files are not present in the GB repository due to space limitations

In [11]:
from SBART.Instruments import ESPRESSO, HARPS
from pathlib import Path


# Either define the path to a text file where each line is a complete path to a S2D file
input_filepath = ""

# or give a list of disk files
input_filepath = [i.as_posix() for i in Path("/home/amiguel/phd/tools/sBART_private/tests/resources").iterdir()]

instrument = ESPRESSO

# Folder in which SBART will store its outputs
storage_path = Path("/home/amiguel/phd/tools/sBART_private/tests/documentation_outputs")


## Configure the pipeline

In [5]:
from SBART.utils.units import meter_second
rv_method = "classical" # Either classical or Laplace

# Define the step that will be used for numerical calculations near max/min points
RVstep = 0.1 * meter_second

# Define the window, around the CCF RV, inside which the models can search for the optimal RV
RV_limits = [200*meter_second, 200*meter_second]


# List with orders to "throw" away
orders_to_skip = []

# Number of cores to use
N_cores = 10


### Configure the different settings of SBART

In [17]:
# For the S2D loading stage
inst_options = {}

# For the creation of the Telluric Model (i.e. the "template generator")
telluric_model_configs = {"CREATION_MODE": "telfit"
                          }

# For the creation of the individual Telluric templates
telluric_template_genesis_configs = {}


# For the creation of the Stellar Model (i.e. the "template generator")

stellar_model_configs = {}

# For the creation of the individual Stellar templates
stellar_template_genesis_configs = {"MINIMUM_NUMBER_OBS": 2
                                    }


confsRV = {"MEMORY_SAVE_MODE": True}




## Setting up the library

By default, SBART's logger is disabled and it will not:

- print information to the terminal
- store a log file to disk

To do so, we must enable the logging interface:

In [7]:
from SBART.outside_tools.create_logger import setup_SBART_logger

setup_SBART_logger(
    storage_path=storage_path / "logs",
    RV_method=rv_method,
    instrument=instrument,
    log_to_terminal=True,
)

2022-05-16T15:46:38 - SBART.outside_tools.create_logger - INFO - /home/amiguel/phd/tools/sBART_private/tests/documentation_outputs/logs


## Loading data

We start by "pointing" sBART towards a few observations:

In [12]:
from SBART.data_objects import DataClassManager

manager = DataClassManager()
manager.start()


data = manager.DataClass(
    input_filepath,
    instrument=instrument,
    instrument_options=inst_options,
)

2022-05-16T15:47:24 - SBART.utils.UserConfigs - DEBUG - Generating internal configs of  - 
2022-05-16T15:47:24 - SBART.data_objects.DataClass - INFO - DataClass opening 2 files from a list/tuple
2022-05-16T15:47:24 - SBART.utils.UserConfigs - DEBUG - Generating internal configs of Frame - ESPRESSO
2022-05-16T15:47:24 - SBART.utils.UserConfigs - DEBUG - Configuration <bypass_QualCheck> using the default value: False
2022-05-16T15:47:24 - SBART.utils.UserConfigs - DEBUG - Configuration <reject_order_percentage> using the default value: 0.25
2022-05-16T15:47:24 - SBART.utils.UserConfigs - DEBUG - Configuration <minimum_order_SNR> using the default value: 20
2022-05-16T15:47:24 - SBART.utils.UserConfigs - DEBUG - Configuration <spectra_format> using the default value: S2D
2022-05-16T15:47:24 - SBART.utils.UserConfigs - DEBUG - Configuration <apply_FluxCorr> using the default value: False
2022-05-16T15:47:24 - SBART.utils.UserConfigs - DEBUG - Configuration <Telluric_Corrected> using the de

2022-05-16T15:47:24 - SBART.Instruments.ESPRESSO - CRITICAL - Did not find the entry for the following UT related metric: HIERARCH ESO INS{} ADC{} RA


2022-05-16T15:47:24 - SBART.Instruments.ESPRESSO - CRITICAL - Did not find the entry for the following UT related metric: HIERARCH ESO INS{} ADC{} dec


2022-05-16T15:47:24 - SBART.Instruments.ESPRESSO - CRITICAL - Did not find the entry for the following UT related metric: HIERARCH ESO INS{} ADC{} dec


2022-05-16T15:47:24 - SBART.Instruments.ESPRESSO - CRITICAL - Did not find the entry for the following UT related metric: HIERARCH ESO INS{} ADC{} SENS1


...

2022-05-16T15:47:24 - SBART.Instruments.ESPRESSO - CRITICAL - Did not find the entry for the following UT related metric: HIERARCH ESO INS{} ADC{} SENS1


2022-05-16T15:47:24 - SBART.Instruments.ESPRESSO - CRITICAL - Did not find the entry for the following UT related metric: HIERARCH ESO INS{} ADC{} SENS1


2022-05-16T15:47:24 - SBART.Instruments.ESPRESSO - CRITICAL - Did not find the entry for the following UT related metric: HIERARCH ESO INS{} ADC{} TEMP


2022-05-16T15:47:24 - SBART.Instruments.ESPRESSO - CRITICAL - Did not find the entry for the following UT related metric: HIERARCH ESO INS{} ADC{} TEMP


2022-05-16T15:47:24 - SBART.Instruments.ESPRESSO - WARNING - Found 1 warning flags in the header KWs
2022-05-16T15:47:24 - SBART.Base_Models.Frame - DEBUG - Frame determined to be from ESPRESSO18
2022-05-16T15:47:24 - SBART.data_objects.DataClass - DEBUG - Selected 2 observations from disk
2022-05-16T15:47:24 - SBART.data_objects.DataClass - INFO - Collecting MetaData from the observations
2022-05-16T15:47:24 - SBART.data_objects.Target - DEBUG - Parsing through loaded OBJECTs
2022-05-16T15:47:24 - SBART.data_objects.Target - INFO - Validated target to be simul
2022-05-16T15:47:24 - SBART.data_objects.DataClass - INFO - Loaded data from KW : DRS-VERSION
2022-05-16T15:47:24 - SBART.data_objects.DataClass - INFO - Loaded data from KW : SPEC_TYPE
2022-05-16T15:47:24 - SBART.data_objects.DataClass - INFO - 
+---------------+-----------+----------------------+-------------+
| subInstrument | Total OBS | Valid OBS [warnings] | INVALID OBS |
+---------------+-----------+----------------------

## Pre-processing the data

Removing activity-sensitive lines

In [13]:
from SBART.Quality_Control.activity_indicators import Indicators

inds = Indicators()
data.remove_activity_lines(inds)

## Creating a telluric model to remove those features

In [14]:
from SBART.template_creation.TelluricModel import TelluricModel

ModelTell = TelluricModel(
    usage_mode="individual",
    user_configs=telluric_model_configs,
    root_folder_path=storage_path,
)

ModelTell.Generate_Model(
    dataClass=data,
    telluric_configs=telluric_template_genesis_configs,
    force_computation=False,
    store_templates=True,
)
data.remove_telluric_features(ModelTell)

2022-05-16T15:47:38 - SBART.Base_Models.TemplateFramework - INFO - Starting Telluric Model
2022-05-16T15:47:38 - SBART.utils.UserConfigs - DEBUG - Generating internal configs of BASE - TemplateFramework
2022-05-16T15:47:38 - SBART.utils.UserConfigs - DEBUG - Configuration <CREATION_MODE> taking the value: telfit
2022-05-16T15:47:38 - SBART.utils.UserConfigs - DEBUG - Configuration <APPLICATION_MODE> using the default value: removal
2022-05-16T15:47:38 - SBART.utils.UserConfigs - DEBUG - Configuration <EXTENSION_MODE> using the default value: lines
2022-05-16T15:47:38 - SBART.template_creation.TelluricModel - INFO - Starting Telluric Model
2022-05-16T15:47:38 - SBART.Base_Models.TemplateFramework - DEBUG - Starting the creation of Telluric models!
2022-05-16T15:47:38 - SBART.Base_Models.TemplateFramework - INFO - Attempting to load previous Templates from disk before creating them
2022-05-16T15:47:38 - SBART.Base_Models.TemplateFramework - DEBUG - Searching in : /home/amiguel/phd/tools/

2022-05-16T15:47:38 - SBART.template_creation.TelluricModel - CRITICAL - Could not find templates to load!


2022-05-16T15:47:38 - SBART.Base_Models.TemplateFramework - INFO - No templates to load from disk. Creating all from scratch
2022-05-16T15:47:38 - SBART.template_creation.TelluricModel - INFO - Using template of type: telfit
2022-05-16T15:47:38 - SBART.utils.UserConfigs - DEBUG - Generating internal configs of Telfit-Telluric from ESPRESSO18
2022-05-16T15:47:38 - SBART.utils.UserConfigs - DEBUG - Configuration <continuum_percentage_drop> using the default value: 1
2022-05-16T15:47:38 - SBART.utils.UserConfigs - DEBUG - Configuration <force_download> using the default value: False
2022-05-16T15:47:38 - SBART.utils.UserConfigs - DEBUG - Configuration <atmosphere_profile> using the default value: download
2022-05-16T15:47:38 - SBART.utils.UserConfigs - DEBUG - Configuration <FIT_MODEL> using the default value: False
2022-05-16T15:47:38 - SBART.utils.UserConfigs - DEBUG - Configuration <FIT_WAVELENGTH_STEP_SIZE> using the default value: 0.001
2022-05-16T15:47:38 - SBART.utils.UserConfigs -

## Creating the stellar templates


In [21]:
from SBART.template_creation.StellarModel import StellarModel

ModelStell = StellarModel(user_configs=stellar_model_configs,
                          root_folder_path=storage_path
                          )

2022-05-16T15:50:35 - SBART.Base_Models.TemplateFramework - INFO - Starting Stellar Model
2022-05-16T15:50:35 - SBART.utils.UserConfigs - DEBUG - Generating internal configs of SpectralModel - Stellar
2022-05-16T15:50:35 - SBART.utils.UserConfigs - DEBUG - Configuration <CREATION_MODE> using the default value: Sum
2022-05-16T15:50:35 - SBART.utils.UserConfigs - DEBUG - Configuration <ALIGNEMENT_RV_SOURCE> using the default value: DRS
2022-05-16T15:50:35 - SBART.utils.UserConfigs - DEBUG - Configuration <PREVIOUS_SBART_PATH> using the default value: 
2022-05-16T15:50:35 - SBART.utils.UserConfigs - DEBUG - Configuration <USE_MERGED_RVS> using the default value: False



When creating the stellar templates we can also reject, temporarily, some observations. They will **not** be used to create
the stellar template, but they will still be used during the RV extraction.

In [22]:
from SBART.utils.spectral_conditions import Empty_condition

StellarTemplateConditions = Empty_condition()

ModelStell.Generate_Model(
        data,
        stellar_template_genesis_configs,
        StellarTemplateConditions,
        force_computation=False,
    )

ModelStell.store_templates_to_disk(storage_path)

data.ingest_StellarModel(ModelStell)


2022-05-16T15:50:38 - SBART.template_creation.StellarModel - INFO - Applying conditions to creation of stellar template
2022-05-16T15:50:38 - SBART.Base_Models.TemplateFramework - DEBUG - Starting the creation of Stellar models!
2022-05-16T15:50:38 - SBART.Base_Models.TemplateFramework - INFO - Attempting to load previous Templates from disk before creating them
2022-05-16T15:50:38 - SBART.Base_Models.TemplateFramework - DEBUG - Searching in : /home/amiguel/phd/tools/sBART_private/tests/documentation_outputs/templates/Stellar/Iteration_0 for Sum
2022-05-16T15:50:38 - SBART.Base_Models.TemplateFramework - INFO - Loading Stellar template of type Sum from disk inside directory
2022-05-16T15:50:38 - SBART.Base_Models.TemplateFramework - INFO - 	/home/amiguel/phd/tools/sBART_private/tests/documentation_outputs/templates/Stellar/Iteration_0
2022-05-16T15:50:38 - SBART.Base_Models.TemplateFramework - INFO - Found 0 available templates: [] of type Sum
2022-05-16T15:50:38 - SBART.Base_Models.Te

  0%|          | 0/2 [00:00<?, ?it/s]

2022-05-16T15:50:39 - SBART.template_creation.stellar_templates.sum_stellar - INFO - Starting frameID: 0


2022-05-16T15:50:40 - SBART.template_creation.stellar_templates.sum_stellar - DEBUG - Frame took 0.921999 seconds
2022-05-16T15:50:40 - SBART.template_creation.stellar_templates.sum_stellar - INFO - Starting frameID: 1
2022-05-16T15:50:41 - SBART.template_creation.stellar_templates.sum_stellar - DEBUG - Frame took 0.771143 seconds
2022-05-16T15:50:41 - SBART.template_creation.stellar_templates.sum_stellar - INFO - Updating template mask
2022-05-16T15:50:41 - SBART.template_creation.stellar_templates.sum_stellar - DEBUG - Ensuring increasing wavelenghs in the stellar template
2022-05-16T15:50:41 - SBART.template_creation.stellar_templates.Stellar_Template - INFO - Computing orders with too many points masked
2022-05-16T15:50:41 - SBART.template_creation.stellar_templates.Stellar_Template - WARNING - Stellar template rejecting order 0 due to having more than 9011/9111 pixels masked
2022-05-16T15:50:41 - SBART.template_creation.stellar_templates.Stellar_Template - WARNING - Stellar templa

2022-05-16T15:50:41 - SBART.template_creation.stellar_templates.Stellar_Template - CRITICAL - ESPRESSO19 has no valid observations. Not computing Stellar template


2022-05-16T15:50:41 - SBART.Base_Models.Template_Model - WARNING - Template will not be created. Check previous error messages
2022-05-16T15:50:41 - SBART.Base_Models.BASE - DEBUG - Setting root path of Sum-Stellar from ESPRESSO19 to /home/amiguel/phd/tools/sBART_private/tests/documentation_outputs/templates/Stellar/Iteration_0
2022-05-16T15:50:41 - SBART.utils.UserConfigs - DEBUG - Generating internal configs of Sum-Stellar from ESPRESSO21
2022-05-16T15:50:41 - SBART.utils.UserConfigs - DEBUG - Configuration <MINIMUM_NUMBER_OBS> taking the value: 2
2022-05-16T15:50:41 - SBART.utils.UserConfigs - DEBUG - Configuration <INTERPOLATION_ERR_PROP> using the default value: interpolation
2022-05-16T15:50:41 - SBART.utils.UserConfigs - DEBUG - Configuration <NUMBER_WORKERS> using the default value: (1, 1)
2022-05-16T15:50:41 - SBART.utils.UserConfigs - DEBUG - Configuration <MEMORY_SAVE_MODE> using the default value: False
2022-05-16T15:50:41 - SBART.utils.UserConfigs - DEBUG - Configuration <

2022-05-16T15:50:41 - SBART.template_creation.stellar_templates.Stellar_Template - CRITICAL - ESPRESSO21 has no valid observations. Not computing Stellar template


2022-05-16T15:50:41 - SBART.Base_Models.Template_Model - WARNING - Template will not be created. Check previous error messages
2022-05-16T15:50:41 - SBART.Base_Models.BASE - DEBUG - Setting root path of Sum-Stellar from ESPRESSO21 to /home/amiguel/phd/tools/sBART_private/tests/documentation_outputs/templates/Stellar/Iteration_0
2022-05-16T15:50:41 - SBART.Base_Models.TemplateFramework - INFO - Storing templates from <Stellar> under the directory
2022-05-16T15:50:41 - SBART.Base_Models.TemplateFramework - INFO - 	/home/amiguel/phd/tools/sBART_private/tests/documentation_outputs/templates/Stellar/Iteration_0
2022-05-16T15:50:41 - SBART.Base_Models.BASE - INFO - Sum-Stellar from ESPRESSO18 storing data to disk
2022-05-16T15:50:41 - SBART.Base_Models.Template_Model - INFO - Storing Sum-Stellar from ESPRESSO18 to disk
2022-05-16T15:50:41 - SBART.Base_Models.Template_Model - WARNING - Disabled removal of old disk files!
2022-05-16T15:50:41 - SBART.template_creation.stellar_templates.Stellar_

## Extracting Radial Velocities

Lastly, we only need to define the RV model that we want to use

In [23]:
from SBART.rv_calculation.RV_Bayesian.RV_Bayesian import RV_Bayesian
from SBART.rv_calculation.rv_stepping.RV_step import RV_step
from SBART.Samplers import chi_squared_sampler, Laplace_approx
import os


if rv_method == "classical":
    sampler = chi_squared_sampler(RVstep, RV_limits)
    rv_model = RV_step(
        processes=N_cores,
        sub_processes=1,
        RV_configs=confsRV,
        sampler=sampler,
    )

    orders = orders_to_skip
elif rv_method == "Laplace":
    sampler = Laplace_approx(RVstep, RV_limits)
    rv_model = RV_Bayesian(
        processes=N_cores,
        sub_processes=1,
        RV_configs=confsRV,
        sampler=sampler,
    )
    orders = os.path.join(storage_path, "Iteration_0/RV_step")
else:
    raise Exception

rv_model.run_routine(data, storage_path, orders)

2022-05-16T15:51:05 - SBART.utils.UserConfigs - DEBUG - Generating internal configs of Sampler - chi_squared
2022-05-16T15:51:05 - SBART.utils.UserConfigs - DEBUG - Configuration <WINDOW_GENERATION_MODE> using the default value: OBSERVATION
2022-05-16T15:51:05 - SBART.utils.UserConfigs - DEBUG - Configuration <STARTING_RV_PIPELINE> using the default value: DRS
2022-05-16T15:51:05 - SBART.utils.UserConfigs - DEBUG - Generating internal configs of  - ModelParameter::RV_component
2022-05-16T15:51:05 - SBART.utils.UserConfigs - DEBUG - Configuration <GENERATION_MODE> taking the value: OBSERVATION
2022-05-16T15:51:05 - SBART.ModelParameters.Model - INFO - Generating model with following available parameters: [Parameter::RV]
2022-05-16T15:51:05 - SBART.utils.UserConfigs - DEBUG - Generating internal configs of RV method - RV_step
2022-05-16T15:51:05 - SBART.utils.UserConfigs - DEBUG - Configuration <MEMORY_SAVE_MODE> taking the value: True
2022-05-16T15:51:05 - SBART.utils.UserConfigs - DEBU

/home/amiguel/.virtualenvs/sbart-xwoIdkmx-py3.8/lib/python3.8/site-packages/astroquery/simbad/core.py:135: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'simul': No known catalog could be found
  warnings.warn("Warning: The script line number %i raised "
2022-05-16T15:51:14 - SBART.data_objects.Target - CRITICAL - Could not compute the secular accelaration from simul
Traceback (most recent call last):

  File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
           │         │     └ {'__name__': '__main__', '__doc__': 'Entry point for launching an IPython kernel.\n\nThis is separate from the ipykernel pack...
           │         └ <code object <module> at 0x7faa280489d0, file "/home/amiguel/.virtualenvs/sbart-xwoIdkmx-py3.8/lib/python3.8/site-packages/ip...
           └ <function _run_code at 0x7faa27fe7310>
  File "/usr/lib/python3.8/runpy.py", l

2022-05-16T15:51:14 - SBART.data_objects.RV_cube - INFO - Setting SA reference frame to BJD = 55500
2022-05-16T15:51:14 - SBART.data_objects.RV_cube - DEBUG - Forcing the drift correction to be set to False
2022-05-16T15:51:14 - SBART.data_objects.RV_cube - DEBUG - Forcing the drift correction to be set to False
2022-05-16T15:51:14 - SBART.data_objects.RV_cube - DEBUG - Forcing the drift correction to be set to False
2022-05-16T15:51:14 - SBART.data_objects.RV_cube - INFO - 
+---------------------+-----------+------------+------------------+
| Method              | std [m/s] | wstd [m/s] | median err [m/s] |
+---------------------+-----------+------------+------------------+
| DRS_raw             | 0.02107   | 0.02107    | 0.00010          |
+---------------------+-----------+------------+------------------+
| DRS_corr            | 0.02107   | 0.02107    | 0.00010          |
+---------------------+-----------+------------+------------------+
| SBART_raw           | 0.00720   | 0.00720 

2022-05-16T15:51:15 - SBART.Base_Models.RV_routine - CRITICAL - 0 - 0


2022-05-16T15:51:15 - SBART.Base_Models.RV_routine - DEBUG - There are 10 live workers. Sending shutdown signal for all
2022-05-16T15:51:15 - SBART.Base_Models.RV_routine - DEBUG - Waiting for worker response
2022-05-16T15:51:15 - SBART.Base_Models.RV_routine - DEBUG - Received 0 shutdown signals. Still missing  10
2022-05-16T15:51:15 - SBART.Base_Models.RV_routine - DEBUG - Received 8 shutdown signals. Still missing  2
2022-05-16T15:51:15 - SBART.Base_Models.RV_routine - DEBUG - Received 0 shutdown signals. Still missing  2
2022-05-16T15:51:15 - SBART.Base_Models.RV_routine - DEBUG - Received 0 shutdown signals. Still missing  2
2022-05-16T15:51:15 - SBART.Base_Models.RV_routine - DEBUG - Received 2 shutdown signals. Still missing  0
2022-05-16T15:51:15 - SBART.Base_Models.RV_routine - INFO - All workers are closed
2022-05-16T15:51:15 - SBART.Base_Models.RV_routine - DEBUG - RV method - RV_step closing multiprocessing interfaces
2022-05-16T15:51:15 - SBART.Base_Models.RV_routine - DEB